# Importing libraries and loading in data

In [25]:
import pandas as pd
from sqlalchemy import create_engine, text
import datetime
import psycopg2

MTA = pd.read_csv("datasets/MTA_monthly_riders.csv")
CTA = pd.read_csv("datasets/CTA_daily_boarding.csv")

## Cleaning Data

In [26]:
MTA = MTA.drop(186)
MTA['Month'] = pd.to_datetime(MTA['Month'])
MTA['city'] = 'New York City'

In [27]:
CTA['Month'] = pd.to_datetime(CTA['service_date'])

## Selecting necessary data and creating 2 new data frames

In [28]:
CTA_refined = pd.melt(CTA, id_vars=['Month'], value_vars=['bus', 'rail_boardings'], var_name='ridership_type', value_name='ridership_count')
CTA_refined.reset_index(drop=True, inplace=True)
CTA_refined = CTA_refined.groupby([CTA_refined['Month'], 'ridership_type']).agg({'ridership_count': 'sum'}).reset_index()
CTA_refined['city'] = 'Chicago'
CTA_refined['ridership_type'] = CTA_refined['ridership_type'].replace('rail_boardings', 'metro')

In [29]:
MTA_refined = MTA[['Month', 'Bus Ridership', 'Subway Ridership', 'city']]
MTA_refined = pd.melt(MTA_refined, id_vars=['Month', 'city'], var_name='ridership_type', value_name='ridership_count')
MTA_refined = MTA_refined.sort_values('Month')
MTA_refined['ridership_type'] = MTA_refined['ridership_type'].replace('Subway Ridership', 'metro')
MTA_refined['ridership_type'] = MTA_refined['ridership_type'].replace('Bus Ridership', 'bus')
MTA_refined['ridership_count'] = MTA_refined['ridership_count'].astype(int)

In [30]:
monthly_totals = pd.concat([MTA_refined, CTA_refined], ignore_index=True)
monthly_totals['Month'] = monthly_totals['Month'].dt.strftime('%Y-%m')

## Creating engine and connecting

In [31]:
host = "localhost"
database = "transportation"
user = "bonks"
password = "bonks"
conn = psycopg2.connect(host=host, database=database, user=user, password=password)

In [32]:
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')

## Putting data into SQL server

In [33]:
monthly_totals.to_sql('monthly_totals', engine, if_exists='replace', index=False)

804

In [34]:
conn.close()

## SQL Commands:
``ALTER TABLE monthly_totals``
``ALTER COLUMN "Month" TYPE date;``
* Makes the date formatting prettier

``CREATE TABLE bus_monthly_totals AS``
``SELECT * FROM monthly_totals WHERE ridership_type = 'bus';``
<br> <br>
``CREATE TABLE metro_monthly_totals AS``
``SELECT * FROM monthly_totals WHERE ridership_type = 'metro';``
* Creates the new tables
    

``ALTER TABLE bus_monthly_totals``
``DROP COLUMN ridership_type;``
<br> <br>
``ALTER TABLE metro_monthly_totals``
``DROP COLUMN ridership_type;``
* Drops `ridership_type` column as it is redundant